In [2]:
import numpy as np
import pandas as pd

from pretrained_embedding import get_input_plus_embedding_vectors
from grid_search_three_subsets import grid_search

from keras.models import Model
from keras.layers import Input, LSTM, Dense, Activation, Bidirectional, Dropout
from keras.layers.embeddings import Embedding

from tensorflow.keras import regularizers

In [3]:
def create_model(vocabulary_length, max_length_sequence, emb_dim, embedding_vectors, lstm_units, 
                 l2_kernel, l2_recurrent, l2_activity, dropout):
    
    X_input = Input(shape = (max_length_sequence, ))
    embedding_layer = Embedding(input_dim = vocabulary_length, output_dim = emb_dim, weights=[embedding_vectors],
                                trainable = False, mask_zero = True)(X_input)
    
    X = LSTM(units = lstm_units, return_sequences = False,
            kernel_regularizer = regularizers.l2(l2_kernel),
            recurrent_regularizer = regularizers.l2(l2_recurrent),
            activity_regularizer = regularizers.l2(l2_activity))(embedding_layer)
    
    X = Dropout(rate = dropout)(X)
    X = Dense(units = 1)(X)
    X = Activation('sigmoid')(X)
                          
    model = Model(inputs = X_input, outputs = X)
                          
    return(model)

In [4]:
english_dataset = pd.read_csv('../data/Merged/english_dataset.csv')
english_dataset

,text,label
0,Donald Trump just couldn t wish all Americans ...,1
1,House Intelligence Committee Chairman Devin Nu...,1
2,"On Friday, it was revealed that former Milwauk...",1
3,"On Christmas day, Donald Trump announced that ...",1
4,Pope Francis used his annual Christmas Day mes...,1
...,...,...
51228,The State Department told the Republican Natio...,0
51229,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,1
51230,Anti-Trump Protesters Are Tools of the Oligar...,1
51231,"ADDIS ABABA, Ethiopia —President Obama convene...",0


In [5]:
vocabulary_length = 10000
max_length_sequence = 1500
emb_dim = 300
language = 'english'
embedding_file_path = '../data/GloVe_Embedding/glove.6B.300d.txt'

In [6]:
X, df, embedding_vectors = get_input_plus_embedding_vectors(english_dataset, embedding_file_path, 
                                                           vocabulary_length, max_length_sequence, emb_dim, language)

In [10]:
model = create_model(vocabulary_length, max_length_sequence, emb_dim, embedding_vectors, 8, 0.01, 0.01, 0, 0.5)
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 1500)              0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 1500, 300)         3000000   
_________________________________________________________________
lstm_5 (LSTM)                (None, 8)                 9888      
_________________________________________________________________
dropout_5 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 9         
_________________________________________________________________
activation_5 (Activation)    (None, 1)                 0         
Total params: 3,009,897
Trainable params: 9,897
Non-trainable params: 3,000,000
_____________________________________________

In [7]:
models = []

model_0 = create_model(vocabulary_length, max_length_sequence, emb_dim, embedding_vectors, lstm_units = 2, l2_kernel = 0.01, l2_recurrent = 0.01, l2_activity = 0, dropout = 0)
model_1 = create_model(vocabulary_length, max_length_sequence, emb_dim, embedding_vectors, lstm_units = 4, l2_kernel = 0.01, l2_recurrent = 0.01, l2_activity = 0, dropout = 0)
model_2 = create_model(vocabulary_length, max_length_sequence, emb_dim, embedding_vectors, lstm_units = 8, l2_kernel = 0.1, l2_recurrent = 0.1, l2_activity = 0, dropout = 0)
model_3 = create_model(vocabulary_length, max_length_sequence, emb_dim, embedding_vectors, lstm_units = 8, l2_kernel = 0, l2_recurrent = 0, l2_activity = 0, dropout = 0.5)

models.append(model_0)
models.append(model_1)
models.append(model_2)
models.append(model_3)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [8]:
Y = df.label.values
epochs = 7
batch_size = 32
iterations = 5
test_size = 0.2

In [11]:
best_model_index, test_acc = grid_search(X, Y, models, epochs, batch_size, iterations, test_size)
print('best_model_index: ' + str(best_model_index), 'test acc: ' + str(test_acc))

Epoch 1/7
32788/32788 [==============================] - 1159s 35ms/step - loss: 0.4413 - accuracy: 0.8723
Epoch 2/7
32788/32788 [==============================] - 1084s 33ms/step - loss: 0.3854 - accuracy: 0.8839
Epoch 3/7
32788/32788 [==============================] - 992s 30ms/step - loss: 0.3151 - accuracy: 0.9060
Epoch 4/7
32788/32788 [==============================] - 965s 29ms/step - loss: 0.3624 - accuracy: 0.8860
Epoch 5/7
32788/32788 [==============================] - 952s 29ms/step - loss: 0.3250 - accuracy: 0.9083
Epoch 6/7
32788/32788 [==============================] - 955s 29ms/step - loss: 0.2947 - accuracy: 0.9119
Epoch 7/7
8198/8198 [==============================] - 124s 15ms/step
Epoch 1/7
32788/32788 [==============================] - 1221s 37ms/step - loss: 0.4817 - accuracy: 0.8489
Epoch 2/7
32788/32788 [==============================] - 1222s 37ms/step - loss: 0.3336 - accuracy: 0.9004
Epoch 3/7
32788/32788 [==============================] - 1037s 32ms/step - los

32788/32788 [==============================] - 1008s 31ms/step - loss: 0.7675 - accuracy: 0.8620
Epoch 2/7
32788/32788 [==============================] - 973s 30ms/step - loss: 0.3825 - accuracy: 0.8874
Epoch 3/7
32788/32788 [==============================] - 977s 30ms/step - loss: 0.4801 - accuracy: 0.8696
Epoch 4/7
32788/32788 [==============================] - 984s 30ms/step - loss: 0.4245 - accuracy: 0.8733
Epoch 5/7
32788/32788 [==============================] - 981s 30ms/step - loss: 0.3840 - accuracy: 0.8761
Epoch 6/7
32788/32788 [==============================] - 1103s 34ms/step - loss: 0.4159 - accuracy: 0.8776
Epoch 7/7
8198/8198 [==============================] - 85s 10ms/step
Epoch 1/7
32788/32788 [==============================] - 983s 30ms/step - loss: 1.2640 - accuracy: 0.7818
Epoch 2/7
32788/32788 [==============================] - 983s 30ms/step - loss: 0.5453 - accuracy: 0.8685
Epoch 3/7
32788/32788 [==============================] - 994s 30ms/step - loss: 0.4319 - ac

40986/40986 [==============================] - 1215s 30ms/step - loss: 0.3061 - accuracy: 0.8896
Epoch 2/7
40986/40986 [==============================] - 1217s 30ms/step - loss: 0.2224 - accuracy: 0.9192
Epoch 3/7
40986/40986 [==============================] - 1281s 31ms/step - loss: 0.1623 - accuracy: 0.9440
Epoch 4/7
40986/40986 [==============================] - 1226s 30ms/step - loss: 0.1495 - accuracy: 0.9514
Epoch 5/7
40986/40986 [==============================] - 1218s 30ms/step - loss: 0.1400 - accuracy: 0.9549
Epoch 6/7
40986/40986 [==============================] - 1212s 30ms/step - loss: 0.1186 - accuracy: 0.9628
Epoch 7/7
40986/40986 [==============================] - 1219s 30ms/step - loss: 0.1140 - accuracy: 0.9665
best_model_index: 3 test acc: 0.924


#### Train Model with English Dataset and Evaluate with Translated Dataset